# Tagging genes with ddlite: candidate extraction

## Introduction
In this example **ddlite** app, we'll build a gene tagger from scratch. Domain-specific tagging systems take months or years to develop. They use hand-crafted model circuitry and accurate, hand-labeled training data. We'll start to build a pretty good one in a few minutes with none of those things. The generalized extraction and learning utilities provided by ddlite will allow us to turn a sampling of article abstracts and some basic domain knowledge into an automated tagging system. Specifically, we want an accurate tagger for genes in academic articles. We have comprehensive dictionaries of genes, but applying a simple matching rule might yield a lot of false positives. For example, "p53" might get tagged as a gene if it refers to a page number. Our goal is to use distant supervision to improve precision.

Here's the pipeline we'll follow:

1. Obtain and parse input data (relevant article abstracts from PubMed)
2. Extract candidates for tagging
3. Generate features
4. Create a test set
5. Write labeling functions
6. Learn the tagging model
7. Iterate on labeling functions

Parts 1 and 2 are covered in this notebook, and parts 3 through 7 are covered in `GeneTaggerExample_Learning.ipynb`. Let's get to it.

In [1]:
%load_ext autoreload
%autoreload 2
import cPickle, os, sys

from snorkel import SnorkelSession
from snorkel.models import Corpus
from snorkel.parser import CorpusParser, HTMLDocParser, SentenceParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Connects to database backend

In [2]:
session = SnorkelSession()

## Processing the input data
We already downloaded the raw HTML for 150 gene-related article pages from PubMed using the `pubmed_gene_html.py` script. These can be found in the `data` folder. We can use ddlite's `DocParser` to read in the article text. There's a general HTML parser which finds visible text, but we can do better by writing a more specific version to just grab the abstract text.

We'll use CoreNLP via ddlite's `SentenceParser` to parse each sentence. `DocParser` can handle this too; we didn't really need that call above. This can take a little while, so if the example has already been run, we'll reload it.

In [3]:
class PubMedAbstractParser(HTMLDocParser):
    def _cleaner(self, s):
        return (s.parent.name == 'abstracttext')

dp = PubMedAbstractParser('gene_tag_example/data/')
cp = CorpusParser(dp, SentenceParser())

In [6]:
corpus = cp.parse_corpus()
corpus.name = 'Pubmed Abstracts'

In [7]:
print len(corpus.documents)
corpus.documents[0]

150


Document('10101268', Corpus (Pubmed Abstracts))

In [8]:
sents = list(corpus.get_sentences())
print len(sents)
sents[0]

1225


Sentence(Document('10101268', Corpus (Pubmed Abstracts)), 0, u'By constructing DNA probes we have identified and cloned a human PtdIns 4-kinase, PI4K230, corresponding to a mRNA of 7.0 kb.')

## Persists corpus in database

In [9]:
session.add(corpus)
session.commit()

## Reloads corpus from database
In case you want to redo candidate extraction without redoing parsing, you can reload the corpus from the database.

In [10]:
corpus = session.query(Corpus).filter(Corpus.name == 'Pubmed Abstracts').one()
corpus

Corpus (Pubmed Abstracts)

## Extracting candidates with matchers
Extracting candidates for mentions (or relations) in ddlite is done with `Matcher` objects. First, we'll use a `DictionaryMatcher`. We have access to a pretty comprehensive gene dictionary. Let's load it in and create the `DictionaryMatcher`.

In [11]:
from snorkel.matchers import DictionaryMatch

# Schema is: ENSEMBL_ID | NAME | TYPE (refseq, canonical, non-canonical)
genes = [line.rstrip().split('\t')[1] for line in open('gene_tag_example/dicts/ensembl_genes.tsv')]
genes = filter(lambda g : len(g) > 2, genes)

gene_dm = DictionaryMatch(d=genes, ignore_case=False, longest_match_only=True)

The dictionary match should provide fairly high recall, but we may still miss some candidates. We know that gene names are named nouns and are often all uppercase. Let's use DDLite's *compositional* matcher operations to handle this. First, we'll write a matcher to find all nouns using the parts-of-speech tags. Then, we'll use a filter to find uppercase sequences. Finally, we'll use a filter to make sure each match has at least 3 characters. We pass `noun_rm` to `up_rm`, and `up_rm` to the final filter to compose them with each other.

In [12]:
from snorkel.matchers import RegexMatchSpan, RegexMatchEach

noun_regex = RegexMatchEach(rgx=r'NN.*', attrib='poses', longest_match_only=True)

up_regex = RegexMatchSpan(
    noun_regex, rgx=r'[A-Z]+([0-9]+)?([A-Z]+)?([0-9]+)?$', ignore_case=False, attrib='words', longest_match_only=True)

multi_regex = RegexMatchSpan(
    up_regex, rgx=r'[a-z0-9]{3,}', ignore_case=True, attrib='words', longest_match_only=True)

Now, we put it all together:

In [13]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import Union

ngrams = Ngrams(n_max=3)

CE = CandidateExtractor(ngrams, Union(gene_dm, multi_regex))

## Creating the candidates
We'll use our unioned candidate extractor to extract our candidate entities from the sentences into a `Candidates` object. Using both matchers together will provide very high recall, but may have poor precision. In the next demo notebook (`GeneTaggerExample_Learning.ipynb`), we'll write distant supervision rules and learn a model to improve precision.

In [14]:
candidates = CE.extract(corpus.get_sentences())
candidates.name = 'Pubmed Candidates'

In [15]:
len(candidates)

1349

Finally, we'll persist the extracted candidates for use in `GeneTaggerExample_Learning.ipynb`.

In [16]:
session.add(candidates)
session.commit()